In [ ]:
import csv
from pathlib import Path
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import spacy
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.cm as cm
from matplotlib.ticker import MultipleLocator
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score

## Transform CSV into DataFrame

In [ ]:
trackers = []
file_names = ["active_tracker.csv", "purged_tracker.csv"]

for file in file_names:
    header_index = None
    with open(file, "r", encoding="utf-8", newline="") as f:
        reader = csv.reader(f)
        for i, row in enumerate(reader):
            if any((cell or "").strip() == "In CRM?" for cell in row):
                header_index = i
                break
    if header_index is None:
        raise ValueError(f"Could not find a row containing 'In CRM?' in any cell of {file}.")
    df = pd.read_csv(file, skiprows=header_index, header=0)
    df = df.dropna(axis=1, how="all")
    df = df.loc[:, ~df.columns.astype(str).str.contains("`")]
    trackers.append(df)

tracker = pd.concat(trackers, ignore_index=True)
tracker = tracker.drop(columns=["General activities", "Project.1"], errors="ignore")
tracker = tracker[~tracker["Status & health"].isin(["DUPLICATE", "LOST"])]
tracker = tracker.drop_duplicates(keep="first").reset_index(drop=True)

# --- Fuzzy match the 'derived/implied' revenue column (KEEP REST UNCHANGED) ---
def _norm(col) -> str:
    s = str(col).lower()
    s = (s.replace("\u00a0", " ")
           .replace("\r", " ")
           .replace("\n", " "))
    return " ".join(s.split())  # collapse whitespace

_tokens_all = ["total", "revenue", "usd"]
_tokens_any = ["derived", "implied"]

_candidates = [
    c for c in tracker.columns
    if all(tok in _norm(c) for tok in _tokens_all) and any(tok in _norm(c) for tok in _tokens_any)
]
if not _candidates:
    raise KeyError("Could not find the 'derived/implied' revenue column by fuzzy match.")
derived = _candidates[0]  # use the first (only) match

def to_number(s: pd.Series) -> pd.Series:
    t = s.astype(str)
    t = t.str.replace("\u00a0", " ", regex=False)     # NBSP -> space
    t = t.str.replace(",", "", regex=False)           # remove commas
    t = t.str.replace("$", "", regex=False)           # remove $
    t = t.str.replace(r"^\((.*)\)$", r"-\1", regex=True)  # (123) -> -123
    # extract first numeric token and convert; no digits -> NaN
    return pd.to_numeric(t.str.extract(r"(-?\d+(?:\.\d+)?)", expand=False), errors="coerce")

# Only use the derived/implied column; no fallback to declared
derived_num = to_number(tracker[derived])

tracker["Total revenue"] = (
    pd.Series(derived_num.where(derived_num > 0, 0), index=tracker.index)
      .fillna(0)
      .astype(int)
)

# --- Diagnostics (entire tracker) ---
# Count rows where the source (derived) is strictly positive
count_pos_source = int((derived_num.notna() & (derived_num > 0)).sum())
# Count rows where Total revenue is strictly positive
count_pos_total  = int((tracker["Total revenue"] > 0).sum())

print("Rows where derived has a positive number (tracker):", count_pos_source)
print("Rows with positive Total revenue (tracker):",        count_pos_total)

# A) derived > 0 but Total revenue == 0  (should be 0 under current rule)
mask_a = (derived_num > 0) & tracker["Total revenue"].eq(0)
if mask_a.any():
    print("\nExamples where derived>0 but Total revenue == 0:")
    print(tracker.loc[mask_a, [derived, "Total revenue"]].head(10))

# B) derived <= 0 or NaN but Total revenue > 0  (should be 0 under current rule)
mask_b = ((derived_num.isna()) | (derived_num <= 0)) & tracker["Total revenue"].gt(0)
if mask_b.any():
    print("\nExamples where derived<=0/NaN but Total revenue > 0:")
    print(tracker.loc[mask_b, [derived, "Total revenue"]].head(10))


In [ ]:
ongoing = tracker[tracker["Status & health"] == "ONGOING"].copy()
complete = tracker[tracker["Status & health"] == "COMPLETE"].copy()
suspended = tracker[tracker["Status & health"] == "SUSPENDED"].copy()

print("Ongoing shape:", ongoing.shape)
print("Completed shape:", complete.shape)
print("Suspended shape:", suspended.shape)


In [ ]:
tracker.columns

In [ ]:
tracker['Product\nline'].unique()

## Revenue Analysis

### Complete - major product lines

In [ ]:
# ----------------- Config -----------------
product_col = 'Product\nline'
date_col    = 'Updated'
value_col   = 'Total revenue'

# Source dataframe expected to exist
sdf = complete.copy()

# ---------- 1) Clean + parse dates robustly ----------
sdf[date_col] = (
    sdf[date_col]
    .astype(str)
    .str.replace("\u00a0", " ", regex=False)
    .str.replace("\r", " ", regex=False)
    .str.replace("\n", " ", regex=False)
    .str.strip()
)
parsed = pd.to_datetime(sdf[date_col], errors="coerce")

# Fallback: expand 2-digit years m/d/yy -> m/d/20yy
mask_na = parsed.isna()
if mask_na.any():
    fixed_2yy = sdf.loc[mask_na, date_col].str.replace(
        r"^(\d{1,2})/(\d{1,2})/(\d{2})$",
        r"\1/\2/20\3",
        regex=True
    )
    parsed.loc[mask_na] = pd.to_datetime(fixed_2yy, errors="coerce")

sdf[date_col] = parsed
sdf = sdf.dropna(subset=[date_col])  # keep only rows with valid dates

# Ensure revenue numeric
sdf[value_col] = pd.to_numeric(sdf[value_col], errors="coerce").fillna(0)

# Drop missing product line rows
sdf = sdf.dropna(subset=[product_col])

# ---------- 2) Map Product line -> Product group by prefix ----------
pl_norm = sdf[product_col].astype(str).str.strip().str.lower()

group = np.select(
    [
        pl_norm.str.startswith('event'),
        pl_norm.str.startswith('viewables'),
        pl_norm.str.startswith('home'),
        pl_norm.str.startswith('others'),
        pl_norm.str.startswith('wearable'),
        pl_norm.str.startswith('hearable'),
    ],
    ['Event', 'Viewables', 'Home', 'Others', 'Wearable', 'Hearable'],
    default='Other'   # fallback for anything else
)
sdf['Product group'] = group

# ---------- 3) Quarter labels & full range ----------
sdf['quarter'] = sdf[date_col].dt.to_period('Q')

if not sdf.empty:
    q_start = sdf['quarter'].min()
    q_end   = sdf['quarter'].max()
    all_quarters = pd.period_range(q_start, q_end, freq='Q')
else:
    all_quarters = pd.PeriodIndex([], freq='Q')

# ---------- 4) Aggregate revenue by quarter x product group ----------
rev_q_pg = (
    sdf.groupby(['quarter', 'Product group'], dropna=False)[value_col]
       .sum()
       .reset_index(name='revenue')
)

# Per-quarter totals and percentages
totals = (rev_q_pg.groupby('quarter', as_index=False)['revenue']
                   .sum()
                   .rename(columns={'revenue': 'q_total'}))
rev_q_pg = rev_q_pg.merge(totals, on='quarter', how='left')
rev_q_pg['pct'] = np.where(
    rev_q_pg['q_total'] > 0,
    100 * rev_q_pg['revenue'] / rev_q_pg['q_total'],
    0.0
)

# ---------- 5) Pivot to % share per group (all groups shown) ----------
pct_pivot = rev_q_pg.pivot_table(
    index='quarter',
    columns='Product group',
    values='pct',
    aggfunc='sum',
    fill_value=0
).reindex(all_quarters, fill_value=0)

# Optional: ensure rows sum to ~100 (floating rounding may lead to 99.999/100.001)
# Not strictly necessary; mapping includes an 'Other' fallback already.

# ---------- 6) Plot stacked % bars (left y-axis) + total revenue line (right y-axis) ----------
quarters_sorted = list(pct_pivot.index.astype(str))
x = np.arange(len(quarters_sorted)) * 1.25
bar_width = 0.8

fig, ax = plt.subplots(figsize=(18, 6))

# Stable color mapping by group name; force "Other" (fallback) to gray
groups = list(pct_pivot.columns)
n_series = len(groups)
cmap = (plt.cm.get_cmap('tab20', n_series) if n_series <= 20 else plt.cm.get_cmap('hsv', n_series))
group_colors = {
    g: ((0.6, 0.6, 0.6, 1.0) if g == 'Other' else cmap(i))
    for i, g in enumerate(groups)
}

# Order stacks by global % contribution for a stable legend/stack order
col_order = pct_pivot.sum(axis=0).sort_values(ascending=False).index.tolist()

bottom = np.zeros(len(pct_pivot), dtype=float)
for g in col_order:
    vals = pct_pivot[g].values
    if np.any(vals != 0):
        ax.bar(x, vals, width=bar_width, bottom=bottom, label=g, color=group_colors[g])
        bottom += vals

# Left y-axis: percentage
ax.set_ylim(0, 100)
ax.set_ylabel("Percent of Total Revenue")

# X-axis formatting
ax.set_xticks(x)
ax.set_xticklabels(quarters_sorted, rotation=45, ha="right")
ax.tick_params(axis="x", labelsize=9, pad=8)
plt.subplots_adjust(bottom=0.25)
ax.margins(x=0.02)

ax.set_title("Share of Revenue and Total Revenue per Quarter")

# Right y-axis: quarter totals as a line
qtot = totals.set_index('quarter')['q_total'].reindex(all_quarters).fillna(0)
ax2 = ax.twinx()
line_plot, = ax2.plot(x, qtot.values, marker='o', linewidth=2, color='#0b1e3f', label="Quarter Total")
ax2.set_ylabel("Total Revenue (Billion USD)")

# Give extra right margin for the legend
fig.subplots_adjust(right=0.78)

# Combined legend (bars + line), placed further right
handles1, labels1 = ax.get_legend_handles_labels()
ax.legend(
    handles1 + [line_plot],
    labels1 + ["Quarter Total"],
    title="Product Lines",
    bbox_to_anchor=(1.05, 1.00),   # farther from plot
    loc="upper left",
    frameon=False
)

plt.tight_layout()
plt.show()

### Complete - specific product lines

In [ ]:
product_col = 'Product\nline'
date_col    = 'Updated'
value_col   = 'Total revenue'

sdf = complete.copy()

# --- 1) Clean + parse dates robustly (handles 1/2-digit M/D and 2- or 4-digit years) ---
sdf[date_col] = (
    sdf[date_col]
    .astype(str)
    .str.replace("\u00a0", " ", regex=False)
    .str.replace("\r", " ", regex=False)
    .str.replace("\n", " ", regex=False)
    .str.strip()
)

# first pass (flexible)
parsed = pd.to_datetime(sdf[date_col], errors="coerce")

# fallback: expand 2-digit years like m/d/yy -> m/d/20yy (e.g., 9/5/25 -> 9/5/2025)
mask_na = parsed.isna()
if mask_na.any():
    fixed_2yy = sdf.loc[mask_na, date_col].str.replace(
        r"^(\d{1,2})/(\d{1,2})/(\d{2})$",
        r"\1/\2/20\3",
        regex=True
    )
    parsed.loc[mask_na] = pd.to_datetime(fixed_2yy, errors="coerce")

sdf[date_col] = parsed
# keep only rows with a valid date
sdf = sdf.dropna(subset=[date_col])

# ensure revenue numeric
sdf[value_col] = pd.to_numeric(sdf[value_col], errors="coerce").fillna(0)

# optional: drop missing product line rows
sdf = sdf.dropna(subset=[product_col])

# --- 2) Quarter labels and completeness over time range ---
sdf['quarter'] = sdf[date_col].dt.to_period('Q')

# sum by quarter x product line
rev_q_pl = (
    sdf.groupby(['quarter', product_col], dropna=False)[value_col]
       .sum()
       .reset_index(name='revenue')
)

# per-quarter totals and percentages
totals = rev_q_pl.groupby('quarter', as_index=False)['revenue'].sum().rename(columns={'revenue':'q_total'})
rev_q_pl = rev_q_pl.merge(totals, on='quarter', how='left')
rev_q_pl['pct'] = np.where(rev_q_pl['q_total'] > 0, 100 * rev_q_pl['revenue'] / rev_q_pl['q_total'], 0.0)

# --- Build a complete PeriodIndex of quarters (unique) ---
if not sdf.empty:
    q_start = sdf['quarter'].min()
    q_end   = sdf['quarter'].max()
    all_quarters = pd.period_range(q_start, q_end, freq='Q')
else:
    all_quarters = pd.PeriodIndex([], freq='Q')

# --- Keep top 3 product lines per quarter by percentage ---
top3 = (
    rev_q_pl.sort_values(['quarter', 'pct'], ascending=[True, False])
            .groupby('quarter', as_index=False)
            .head(3)
)

# === Build PERCENT stacks: top-3 % + "Other" so each bar reaches 100% ===
pct_pivot = top3.pivot_table(
    index='quarter',
    columns=product_col,
    values='pct',        # percentages
    aggfunc='sum',
    fill_value=0
)

# Ensure all quarters present
pct_pivot = pct_pivot.reindex(all_quarters, fill_value=0)

# Add "Other" percentage per quarter to reach 100
row_sum = pct_pivot.sum(axis=1).clip(0, 100)
pct_pivot['Other'] = (100 - row_sum).clip(lower=0, upper=100)

# Order legend/stack by global contribution (sum of % across quarters)
col_order = pct_pivot.sum(axis=0).sort_values(ascending=False).index.tolist()
# Put "Other" at the end for readability
if 'Other' in col_order:
    col_order = [c for c in col_order if c != 'Other'] + ['Other']

# --- Plot stacked bars in PERCENTAGES (left y-axis) ---
quarters_sorted = list(pct_pivot.index.astype(str))

spacing = 1.25
x = np.arange(len(quarters_sorted)) * spacing
bar_width = 0.8

fig, ax = plt.subplots(figsize=(18, 6))

# Colors; force "Other" to gray
n_series = len(col_order)
cmap = (plt.cm.get_cmap('tab20', n_series) if n_series <= 20
        else plt.cm.get_cmap('hsv', n_series))
series_colors = {name: (0.6, 0.6, 0.6, 1.0) if name == 'Other' else cmap(i)
                 for i, name in enumerate(col_order)}

bottom = np.zeros(len(pct_pivot), dtype=float)
for name in col_order:
    vals = pct_pivot[name].values
    if np.any(vals != 0):
        ax.bar(x, vals, width=bar_width, bottom=bottom, label=name, color=series_colors[name])
        bottom += vals

# Left y-axis: percentage
ax.set_ylim(0, 100)
ax.set_ylabel("Percent of Total Revenue")

# X-axis labels
ax.set_xticks(x)
ax.set_xticklabels(quarters_sorted, rotation=45, ha="right")
ax.tick_params(axis="x", labelsize=9, pad=8)
plt.subplots_adjust(bottom=0.25)
ax.margins(x=0.02)

ax.set_title("Share of Revenue and Total Revenue per Quarter")

# --- Overlay line of quarter totals on RIGHT y-axis ---
qtot = totals.set_index('quarter')['q_total'].reindex(all_quarters).fillna(0)
ax2 = ax.twinx()
# Dark navy line
line_plot, = ax2.plot(x, qtot.values, marker='o', linewidth=2, color='#0b1e3f', label="Quarter Total")
ax2.set_ylabel("Total Revenue (Billion USD)")

# Give extra right margin for the farther legend
fig.subplots_adjust(right=0.78)

# --- Combined legend (bars + line), moved farther right ---
handles1, labels1 = ax.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
ax.legend(
    handles1 + [line_plot],
    labels1 + ["Quarter Total"],
    title="Product Lines",
    bbox_to_anchor=(1.05, 1.00),
    loc="upper left",
    frameon=False
)


### Complete - specific BUs

In [ ]:
product_col = 'Owner unit'
date_col    = 'Updated'
value_col   = 'Total revenue'

sdf = complete.copy()

# --- 1) Clean + parse dates robustly (handles 1/2-digit M/D and 2- or 4-digit years) ---
sdf[date_col] = (
    sdf[date_col]
    .astype(str)
    .str.replace("\u00a0", " ", regex=False)
    .str.replace("\r", " ", regex=False)
    .str.replace("\n", " ", regex=False)
    .str.strip()
)

# first pass (flexible)
parsed = pd.to_datetime(sdf[date_col], errors="coerce")

# fallback: expand 2-digit years like m/d/yy -> m/d/20yy (e.g., 9/5/25 -> 9/5/2025)
mask_na = parsed.isna()
if mask_na.any():
    fixed_2yy = sdf.loc[mask_na, date_col].str.replace(
        r"^(\d{1,2})/(\d{1,2})/(\d{2})$",
        r"\1/\2/20\3",
        regex=True
    )
    parsed.loc[mask_na] = pd.to_datetime(fixed_2yy, errors="coerce")

sdf[date_col] = parsed
# keep only rows with a valid date
sdf = sdf.dropna(subset=[date_col])

# ensure revenue numeric
sdf[value_col] = pd.to_numeric(sdf[value_col], errors="coerce").fillna(0)

# optional: drop missing owner unit rows
sdf = sdf.dropna(subset=[product_col])

# --- 2) Quarter labels and completeness over time range ---
sdf['quarter'] = sdf[date_col].dt.to_period('Q')

# sum by quarter x owner unit
rev_q_pl = (
    sdf.groupby(['quarter', product_col], dropna=False)[value_col]
       .sum()
       .reset_index(name='revenue')
)

# per-quarter totals and percentages
totals = rev_q_pl.groupby('quarter', as_index=False)['revenue'].sum().rename(columns={'revenue':'q_total'})
rev_q_pl = rev_q_pl.merge(totals, on='quarter', how='left')
rev_q_pl['pct'] = np.where(rev_q_pl['q_total'] > 0, 100 * rev_q_pl['revenue'] / rev_q_pl['q_total'], 0.0)

# --- Build a complete PeriodIndex of quarters (unique) ---
if not sdf.empty:
    q_start = sdf['quarter'].min()
    q_end   = sdf['quarter'].max()
    all_quarters = pd.period_range(q_start, q_end, freq='Q')
else:
    all_quarters = pd.PeriodIndex([], freq='Q')

# --- Keep top 3 owner units per quarter by percentage ---
top3 = (
    rev_q_pl.sort_values(['quarter', 'pct'], ascending=[True, False])
            .groupby('quarter', as_index=False)
            .head(3)
)

# === Build PERCENT stacks: top-3 % + "Other" so each bar reaches 100% ===
pct_pivot = top3.pivot_table(
    index='quarter',
    columns=product_col,   # Owner unit
    values='pct',
    aggfunc='sum',
    fill_value=0
)

# Ensure all quarters present (even if 0%)
pct_pivot = pct_pivot.reindex(all_quarters, fill_value=0)

# Add "Other" percentage per quarter to reach 100
row_sum = pct_pivot.sum(axis=1)
pct_pivot['Other'] = (100 - row_sum).clip(lower=0, upper=100)

# Order stacks by global contribution; put "Other" last for readability
col_order = pct_pivot.sum(axis=0).sort_values(ascending=False).index.tolist()
if 'Other' in col_order:
    col_order = [c for c in col_order if c != 'Other'] + ['Other']

# --- Plot stacked bars in PERCENTAGES (left y-axis) ---
quarters_sorted = list(pct_pivot.index.astype(str))

spacing = 1.25
x = np.arange(len(quarters_sorted)) * spacing
bar_width = 0.8

fig, ax = plt.subplots(figsize=(18, 6))

# Stable colors by unit name; force "Other" to gray
units = list(pct_pivot.columns)
n_series = len(units)
cmap = (plt.cm.get_cmap('tab20', n_series) if n_series <= 20 else plt.cm.get_cmap('hsv', n_series))
unit_colors = {u: ((0.6, 0.6, 0.6, 1.0) if u == 'Other' else cmap(i)) for i, u in enumerate(units)}

bottom = np.zeros(len(pct_pivot), dtype=float)
for col in col_order:
    vals = pct_pivot[col].values
    if np.any(vals != 0):
        ax.bar(x, vals, width=bar_width, bottom=bottom, label=col, color=unit_colors[col])
        bottom += vals

# Left y-axis: percentage
ax.set_ylim(0, 100)
ax.set_ylabel("Percent of Total Revenue")

# X-axis labels
ax.set_xticks(x)
ax.set_xticklabels(quarters_sorted, rotation=45, ha="right")
ax.tick_params(axis="x", labelsize=9, pad=8)
plt.subplots_adjust(bottom=0.25)
ax.margins(x=0.02)

ax.set_title("Share of Revenue and Total Revenue per Quarter")

# --- Overlay line of quarter totals on RIGHT y-axis (dark navy) ---
qtot = totals.set_index('quarter')['q_total'].reindex(all_quarters).fillna(0)
ax2 = ax.twinx()
line_plot, = ax2.plot(x, qtot.values, marker='o', linewidth=2, color='#0b1e3f', label="Quarter Total")
ax2.set_ylabel("Total Revenue (Billion USD)")

# Give extra right margin for the farther legend
fig.subplots_adjust(right=0.78)

# --- Combined legend (bars + line), moved further right ---
handles1, labels1 = ax.get_legend_handles_labels()
ax.legend(
    handles1 + [line_plot],
    labels1 + ["Quarter Total"],
    title="BU",
    bbox_to_anchor=(1.05, 1.00),  # farther from plot
    loc="upper left",
    frameon=False
)

plt.tight_layout()
plt.show()

### Ongoing - major product lines

In [ ]:
# --- Config ---
product_col = 'Product\nline'
value_col   = 'Total revenue'
threshold_pct = 3.0  # group categories contributing < 3% into "Other"

# --- 1) Prep & aggregate ---
og = ongoing.copy()
og[value_col] = pd.to_numeric(og[value_col], errors="coerce").fillna(0)

# --- 1a) Map Product line -> Product group by prefix ---
pl_norm = og[product_col].astype(str).str.strip().str.lower()

is_event     = pl_norm.str.startswith('event')
is_viewables = pl_norm.str.startswith('viewables')
is_home      = pl_norm.str.startswith('home')
is_wearable  = pl_norm.str.startswith('wearable')
is_hearable  = pl_norm.str.startswith('hearable')
is_others    = pl_norm.str.startswith('others')  # treat explicit "others" as Other

og['Product group'] = np.select(
    [is_event, is_viewables, is_home, is_wearable, is_hearable, is_others],
    ['Event',  'Viewables',  'Home', 'Wearable', 'Hearable',  'Other'],
    default='Other'  # fallback for anything else (incl. NaN after astype(str))
)

group_col = 'Product group'  # use groups for the pie

# --- 1b) Aggregate by group (not by individual product lines) ---
by_group = (
    og.groupby(group_col, dropna=False)[value_col]
      .sum()
      .sort_values(ascending=False)
)

total_revenue = float(by_group.sum())
print(f"Total revenue (ongoing): {total_revenue:,.0f}")

if total_revenue == 0:
    print("No revenue in 'ongoing' to plot.")
else:
    # --- 2) Percentages & thresholding (avoid double 'Other') ---
    pct = (by_group / total_revenue) * 100.0
    base_other = float(by_group.get('Other', 0.0))

    keep_mask = (pct >= threshold_pct) | (by_group.index == 'Other')
    kept = by_group[keep_mask]
    small_sum = by_group[~keep_mask].sum()

    # Merge small categories into 'Other'
    others_value = base_other + small_sum
    final = kept.drop(index='Other', errors='ignore')
    if others_value > 0:
        final = pd.concat([final, pd.Series({'Other': others_value})])

    # Ensure descending order for legend consistency
    final = final.sort_values(ascending=False)

    # --- 3) Unique colors (no reuse) ---
    n = len(final)
    cmap = plt.cm.get_cmap('tab20', n) if n <= 20 else plt.cm.get_cmap('hsv', n)
    colors = [cmap(i) for i in range(n)]

    # --- 3a) Two-line labels in millions; hide *label text* for 'Other'
    group_names = list(final.index)
    labels_two_line = [
        (f"{name}({val/1e6:.1f}M)" if name != 'Other' else "")
        for name, val in final.items()
    ]

    # --- 3b) autopct callable that *skips* percent for 'Other'
    def autopct_excluding_other(names):
        def fmt(pct):
            fmt.i += 1
            idx = fmt.i - 1
            return "" if names[idx] == 'Other' else f"{pct:.1f}%"
        fmt.i = 0
        return fmt

    # --- 4) Pie chart (original style, with 'Other' unlabeled) ---
    fig, ax = plt.subplots(figsize=(9, 9))
    wedges, texts, autotexts = ax.pie(
        final.values,
        labels=labels_two_line,                          # blank label for 'Other'
        autopct=autopct_excluding_other(group_names),    # blank % for 'Other'
        startangle=90,
        counterclock=False,
        colors=colors,
        labeldistance=1.25,                              # small nudge outward
        pctdistance=0.72,                                # % a bit toward center
        textprops={"fontsize": 10}
    )

    ax.axis('equal')
    ax.set_title("Potential Revenue by Main Product Lines")

    # --- 5) Legend: include ALL groups (including 'Other') ---
    plt.subplots_adjust(right=0.58)  # leave more empty space on the right
    ax.legend(
        handles=wedges,
        labels=list(final.index),     # include 'Other' in legend
        title="Product Line",
        loc="center left",
        bbox_to_anchor=(1.50, 0.5),   # move legend farther from the axes
        frameon=True,
        borderaxespad=0.0
    )

    plt.show()





### Ongoing - specific product lines

In [ ]:
# --- Config ---
product_col = 'Product\nline'
value_col   = 'Total revenue'
threshold_pct = 3.0  # group categories contributing < 3% into "Other"

# --- 1) Prep & aggregate ---
og = ongoing.copy()
og[value_col] = pd.to_numeric(og[value_col], errors="coerce").fillna(0)

by_pl = (
    og.groupby(product_col, dropna=False)[value_col]
      .sum()
      .sort_values(ascending=False)
)
by_pl.index = by_pl.index.fillna("Unknown")

total_revenue = float(by_pl.sum())
print(f"Total revenue (ongoing): {total_revenue:,.0f}")

if total_revenue == 0:
    print("No revenue in 'ongoing' to plot.")
else:
    # --- 2) Percentages & thresholding ---
    pct = (by_pl / total_revenue) * 100.0
    keep_mask = pct >= threshold_pct
    kept = by_pl[keep_mask]
    others_value = by_pl[~keep_mask].sum()

    # Build final series with "Other" group if needed
    if others_value > 0:
        final = pd.concat([kept, pd.Series({"Other": others_value})])
    else:
        final = kept.copy()

    # Ensure descending order for legend consistency
    final = final.sort_values(ascending=False)

    # --- 3) Unique colors (no reuse) ---
    n = len(final)
    cmap = plt.cm.get_cmap('tab20', n) if n <= 20 else plt.cm.get_cmap('hsv', n)
    colors = [cmap(i) for i in range(n)]

    # Build labels as "Name (xx.xM)" where values are in millions
    labels_with_values = [f"{name} ({val/1e6:.1f}M)" for name, val in final.items()]

    # --- 4) Pie chart ---
    fig, ax = plt.subplots(figsize=(8, 8))
    wedges, texts, autotexts = ax.pie(
        final.values,
        labels=labels_with_values,          
        autopct=lambda p: f"{p:.1f}%",
        startangle=90,
        counterclock=False,
        colors=colors
    )

    ax.axis('equal')
    ax.set_title("Potential Revenue by Specific Product Lines")

    # --- 5) Legend: ONLY color and product line name ---
    plt.subplots_adjust(right=0.58)  # leave more empty space on the right
    ax.legend(
        handles=wedges,
        labels=list(final.index),     # only color + product line name
        title="Product line",
        loc="center left",
        bbox_to_anchor=(1.70, 0.5),  # move legend farther from the axes (increase to push further)
        frameon=True,
        borderaxespad=0.0
    )

    plt.show()


### Ongoing - BUs

In [ ]:
# --- Config ---
group_col = 'Owner unit'      # grouping column
value_col = 'Total revenue'   # numeric values
threshold_pct = 3.0           # group categories contributing < 3% into "Other"

# --- 1) Prep & aggregate ---
og = ongoing.copy()
og[value_col] = pd.to_numeric(og[value_col], errors="coerce").fillna(0)

by_group = (
    og.groupby(group_col, dropna=False)[value_col]
      .sum()
      .sort_values(ascending=False)
)
by_group.index = by_group.index.fillna("Unknown")

total_revenue = float(by_group.sum())
print(f"Total revenue (ongoing): {total_revenue:,.0f}")

if total_revenue == 0:
    print("No revenue in 'ongoing' to plot.")
else:
    # --- 2) Percentages & thresholding ---
    pct = (by_group / total_revenue) * 100.0
    keep_mask = pct >= threshold_pct
    kept = by_group[keep_mask]
    others_value = by_group[~keep_mask].sum()

    # Build final series with "Other" bucket if needed
    if others_value > 0:
        final = pd.concat([kept, pd.Series({"Other": others_value})])
    else:
        final = kept.copy()

    # Order for consistent legend appearance
    final = final.sort_values(ascending=False)

    # --- 3) Unique colors (no reuse) ---
    n = len(final)
    cmap = plt.cm.get_cmap('tab20', n) if n <= 20 else plt.cm.get_cmap('hsv', n)
    colors = [cmap(i) for i in range(n)]

    # Build labels as "Name (xx.xM)" where values are in millions
    labels_with_values = [f"{name} ({val/1e6:.1f}M)" for name, val in final.items()]

    # --- 4) Pie chart (labels spaced farther apart) ---
    fig, ax = plt.subplots(figsize=(8, 8))
    wedges, texts, autotexts = ax.pie(
        final.values,
        labels=labels_with_values,          # ← use name + total revenue in millions
        autopct=lambda p: f"{p:.1f}%",
        startangle=90,
        counterclock=False,
        colors=colors,
        labeldistance=1.3,
        pctdistance=0.7,
        textprops={"fontsize": 10}
    )

    ax.axis('equal')
    ax.set_title("Potential Revenue by Business Units")

    # --- 5) Legend: ONLY color + name, placed farther from the pie ---
    plt.subplots_adjust(right=0.58)  # reserve more space on the right
    ax.legend(
        handles=wedges,
        labels=list(final.index),
        title="BU",
        loc="center left",
        bbox_to_anchor=(1.45, 0.5),
        frameon=True,
        borderaxespad=0.0
    )

    plt.show()


In [ ]:
# Robust numeric parser (same as before)
def to_number(s: pd.Series) -> pd.Series:
    t = s.astype(str)
    t = t.str.replace("\u00a0", " ", regex=False)     # NBSP -> space
    t = t.str.replace(",", "", regex=False)           # remove commas
    t = t.str.replace("$", "", regex=False)           # remove $
    t = t.str.replace(r"^\((.*)\)$", r"-\1", regex=True)  # (123) -> -123
    # extract first numeric token and convert
    return pd.to_numeric(t.str.extract(r"(-?\d+(?:\.\d+)?)", expand=False), errors="coerce")

# --- Parse numbers from sources on the WHOLE tracker ---
declared_num = to_number(suspended[declared])
derived_num  = to_number(suspended[derived])

# 1) Mask-level equivalence (your test)
mask_sources_nonzero = (declared_num.fillna(0) != 0) | (derived_num.fillna(0) != 0)
mask_total_nonzero   = suspended["Total revenue"].fillna(0) != 0
assert (mask_sources_nonzero == mask_total_nonzero).all(), "Mismatch: non-zero masks differ"

# 2) Value-level equivalence (exact numbers with your precedence rule)
expected = np.where(declared_num.notna() & (declared_num != 0), declared_num,
            np.where(derived_num.notna()  & (derived_num  != 0), derived_num, 0))
expected = pd.Series(expected, index=suspended.index).fillna(0).astype(int)
assert suspended["Total revenue"].equals(expected), "Mismatch: values differ from precedence rule"
print("Diagnostics passed: masks and values align.")

 ## Opportunity Analysis

### Number of Opportunities

In [ ]:
# --- Columns ---
prod_col    = 'Product\nline'
time_col = 'Created'
stage_col   = 'Stage'

# --- 1) Make a working copy ---
df = tracker.copy()

# --- 2) Map Product line to top-level group by prefix ---
# Normalize for prefix checks
pl = df[prod_col].astype(str).str.strip().str.lower()

group = np.select(
    [
        pl.str.startswith('event'),
        pl.str.startswith('viewables'),
        pl.str.startswith('home'),
        pl.str.startswith('others'),
        pl.str.startswith('wearable'),
        pl.str.startswith('hearable'),
    ],
    ['Event', 'Viewables', 'Home', 'Others', 'Wearable', 'Hearable'],
    default='Other'  # fallback if no known prefix
)
df['Product group'] = group

# (Optional) if you want to drop 'Other' (fallbacks), uncomment:
# df = df[df['Product group'] != 'Other']

# --- 3) Parse Created -> datetime robustly, then to quarter ---
clean_created = (
    df[time_col].astype(str)
      .str.replace("\u00a0", " ", regex=False)
      .str.replace("\r", " ", regex=False)
      .str.replace("\n", " ", regex=False)
      .str.strip()
)

parsed = pd.to_datetime(clean_created, errors='coerce')  # flexible first pass

# Fallback: convert m/d/yy to m/d/20yy (e.g., 9/5/25 -> 9/5/2025)
mask_na = parsed.isna()
if mask_na.any():
    fixed_2yy = clean_created[mask_na].str.replace(
        r"^(\d{1,2})/(\d{1,2})/(\d{2})$",
        r"\1/\2/20\3",
        regex=True
    )
    parsed.loc[mask_na] = pd.to_datetime(fixed_2yy, errors='coerce')

df['Created_dt'] = parsed
df = df.dropna(subset=['Created_dt'])  # keep rows with valid dates
df['quarter'] = df['Created_dt'].dt.to_period('Q')

# --- 4) Filter to rows where Stage contains "opportunity" (case-insensitive) ---
is_oppty = df[stage_col].astype(str).str.contains('opportunity', case=False, na=False)
df_oppty = df.loc[is_oppty, ['quarter', 'Product group']].copy()

# --- 5) Count opportunities per quarter x product group ---
counts = (
    df_oppty.groupby(['quarter', 'Product group'])
            .size()
            .unstack('Product group', fill_value=0)
)

# Ensure continuous quarters on the index (even if zero)
if not counts.empty:
    all_quarters = pd.period_range(counts.index.min(), counts.index.max(), freq='Q')
    counts = counts.reindex(all_quarters, fill_value=0)

# Optional: order columns by total volume (largest first)
col_order = counts.sum(axis=0).sort_values(ascending=False).index.tolist()
counts = counts[col_order]

# --- 6) Plot: lines per product group over quarters ---
quarters = list(counts.index.astype(str))
x = np.arange(len(quarters))

fig, ax = plt.subplots(figsize=(14, 6))

# Use a categorical colormap with enough distinct colors
n_series = counts.shape[1]
cmap = plt.cm.get_cmap('tab20', n_series) if n_series <= 20 else plt.cm.get_cmap('hsv', n_series)
colors = [cmap(i) for i in range(n_series)]

line_width = 3
marker_size = 6

for i, col in enumerate(counts.columns):
    ax.plot(
        x,
        counts[col].values,
        label=col,
        linewidth=line_width,   # thicker
        marker='o',
        markersize=marker_size,
        color=colors[i]
    )

ax.set_xticks(x)
ax.set_xticklabels(quarters, rotation=45, ha='right')
ax.set_ylabel('Number of opportunities')
ax.set_title('Quarterly Opportunities by Main Product Lines')
ax.grid(True, axis='y', alpha=0.3)
# Force integer y-axis ticks with step = 1
y_max = int(np.nanmax(counts.values)) if counts.size else 0
ax.set_ylim(0, max(1, y_max))
ax.yaxis.set_major_locator(MultipleLocator(1))
ax.margins(x=0.02)

# Legend outside (optional). Comment these two lines to keep legend inside.
plt.subplots_adjust(right=0.82)
ax.legend(title='Product Line', loc='center left', bbox_to_anchor=(1.02, 0.5), frameon=True)

plt.show()

### Average Time Spent on Suspended Opportunities

In [ ]:
# --- Columns ---
product_col = 'Product\nline'
stage_col   = 'Stage'
time_col    = 'Time\n(days)'

# --- 1) Copy & basic cleaning ---
df = suspended.copy()
df[time_col] = pd.to_numeric(df[time_col], errors='coerce')  # ensure numeric

# --- 2) Filter to Stage contains "opportunity" (case-insensitive) ---
is_oppty = df[stage_col].astype(str).str.contains('opportunity', case=False, na=False)
df_oppty = df.loc[is_oppty].copy()

if df_oppty.empty:
    print("No rows in 'suspended' with Stage containing 'opportunity'.")
else:
    # --- 3) Map Product line -> Product group by prefix ---
    pl_norm = df_oppty[product_col].astype(str).str.strip().str.lower()

    is_event     = pl_norm.str.startswith('event')
    is_viewables = pl_norm.str.startswith('viewables')
    is_home      = pl_norm.str.startswith('home')
    is_wearable  = pl_norm.str.startswith('wearable')
    is_hearable  = pl_norm.str.startswith('hearable')
    is_others    = pl_norm.str.startswith('others')  # explicit "others"

    df_oppty['Product group'] = np.select(
        [is_event, is_viewables, is_home, is_wearable, is_hearable, is_others],
        ['Event',  'Viewables',  'Home', 'Wearable', 'Hearable',  'Other'],
        default='Other'  # fallback for anything else (incl. NaN after astype(str))
    )

    # --- 4) Compute average Time(days) per Product group ---
    group_means = (
        df_oppty.groupby('Product group', dropna=False)[time_col]
                .mean()
                .dropna()
                .sort_values(ascending=False)
    )

    if group_means.empty:
        print("No valid 'Time\\n(days)' values after filtering.")
    else:
        # --- 5) Bar plot (horizontal) ---
        y = np.arange(len(group_means))
        vals = group_means.values
        labels = group_means.index.tolist()

        fig, ax = plt.subplots(figsize=(10, 6))
        ax.barh(y, vals)                 # no custom colors per your style constraints
        ax.set_yticks(y)
        ax.set_yticklabels(labels)
        ax.invert_yaxis()                # largest at top

        ax.set_xlabel("Number of Days")
        ax.set_title("Average Time Spent on Opportunities by Main Product Lines")

        # Optional: annotate values on bars
        for yi, v in zip(y, vals):
            ax.text(v, yi, f"  {v:.1f}", va="center")

        plt.tight_layout()
        plt.show()

In [ ]:
col = 'Time\n(days)'
prod_col = 'Product\nline'

# numeric series for sorting
s = pd.to_numeric(tracker[col], errors='coerce')

# top 5 rows by value (keeps duplicates)
top5_idx = s.nlargest(5).index

# grab corresponding product lines + numeric time
top5 = tracker.loc[top5_idx, [prod_col]].copy()
top5['Time (days)'] = s.loc[top5_idx].values  # ensure numeric

# optional: keep the exact original order (already descending)
print(top5)

## Quote Analysis

### Main Product Lines

In [ ]:
# --- Columns ---
prod_col     = 'Product\nline'
time_col  = 'Updated'
flag_cols    = ['RFI', 'RFP', 'RFQ', 'SOW']

# --- 1) Working copy ---
df = tracker.copy()

# --- 2) Map Product line to 6 top-level groups by prefix ---
pl_norm = df[prod_col].astype(str).str.strip().str.lower()
groups = np.select(
    [
        pl_norm.str.startswith('event'),
        pl_norm.str.startswith('viewables'),
        pl_norm.str.startswith('home'),
        pl_norm.str.startswith('others'),
        pl_norm.str.startswith('wearable'),
        pl_norm.str.startswith('hearable'),
    ],
    ['Event', 'Viewables', 'Home', 'Others', 'Wearable', 'Hearable'],
    default=np.nan  # drop anything that doesn't match these 6 prefixes
)
df['Product group'] = groups
df = df.dropna(subset=['Product group'])

# --- 3) Parse Created -> datetime robustly, then to quarter ---
time_clean = (
    df[time_col].astype(str)
      .str.replace("\u00a0", " ", regex=False)
      .str.replace("\r", " ", regex=False)
      .str.replace("\n", " ", regex=False)
      .str.strip()
)

created_dt = pd.to_datetime(time_clean, errors='coerce')
mask_na = created_dt.isna()
if mask_na.any():
    # fallback: m/d/yy -> m/d/20yy
    fixed_2yy = time_clean[mask_na].str.replace(
        r"^(\d{1,2})/(\d{1,2})/(\d{2})$",
        r"\1/\2/20\3",
        regex=True
    )
    created_dt.loc[mask_na] = pd.to_datetime(fixed_2yy, errors='coerce')

df['Created_dt'] = created_dt
df = df.dropna(subset=['Created_dt'])
df['quarter'] = df['Created_dt'].dt.to_period('Q')

# --- 4) Keep rows where at least one of the flags contains "Yes" (case-insensitive) ---
present_flags = [c for c in flag_cols if c in df.columns]
if not present_flags:
    raise KeyError(f"None of the flag columns are present: {flag_cols}")

yes_any = (
    df[present_flags]
      .astype(str)
      .apply(lambda col: col.str.contains(r'\byes\b', case=False, na=False))
      .any(axis=1)
)
df_yes = df.loc[yes_any, ['quarter', 'Product group']].copy()

# --- 5) Count per quarter x product group ---
counts = (
    df_yes.groupby(['quarter', 'Product group'])
          .size()
          .unstack('Product group', fill_value=0)
)

# Ensure continuous quarter index (even if no rows in some quarters)
if not counts.empty:
    all_quarters = pd.period_range(counts.index.min(), counts.index.max(), freq='Q')
    counts = counts.reindex(all_quarters, fill_value=0)

# Order groups by total volume (largest first) for a tidy legend
col_order = counts.sum(axis=0).sort_values(ascending=False).index.tolist()
counts = counts[col_order]

# --- 6) Plot: lines per product group over quarters ---
quarters = list(counts.index.astype(str))
x = np.arange(len(quarters))

fig, ax = plt.subplots(figsize=(14, 6))

# distinct colors; thicker lines
n_series = counts.shape[1]
cmap = plt.cm.get_cmap('tab20', n_series) if n_series <= 20 else plt.cm.get_cmap('hsv', n_series)
colors = [cmap(i) for i in range(n_series)]

line_width = 3
marker_size = 6
for i, col in enumerate(counts.columns):
    ax.plot(
        x,
        counts[col].values,
        label=col,
        linewidth=line_width,
        marker='o',
        markersize=marker_size,
        color=colors[i]
    )

ax.set_xticks(x)
ax.set_xticklabels(quarters, rotation=45, ha='right')
ax.set_ylabel('Number of Cases with A Quote')
ax.set_title('RFI/RFP/RFQ/SOW per Quarter by Main Product Lines')
ax.grid(True, axis='y', alpha=0.3)
ax.margins(x=0.02)

# Legend outside (optional)
plt.subplots_adjust(right=0.82, bottom=0.18)
ax.legend(title='Product Line', loc='center left', bbox_to_anchor=(1.02, 0.5), frameon=True)

plt.show()


### BUs

In [ ]:
# --- Columns ---
owner_col   = "Owner unit"
updated_col = "Updated"
flag_cols   = ["RFI", "RFP", "RFQ", "SOW"]

# --- 1) Working copy & robust date parsing for Updated -> quarter ---
df = tracker.copy()

clean_upd = (
    df[updated_col].astype(str)
      .str.replace("\u00a0", " ", regex=False)
      .str.replace("\r", " ", regex=False)
      .str.replace("\n", " ", regex=False)
      .str.strip()
)

upd_dt = pd.to_datetime(clean_upd, errors="coerce")

# Fallback: m/d/yy -> m/d/20yy (e.g., 9/5/25 -> 9/5/2025)
mask_na = upd_dt.isna()
if mask_na.any():
    fixed_2yy = clean_upd[mask_na].str.replace(
        r"^(\d{1,2})/(\d{1,2})/(\d{2})$",
        r"\1/\2/20\3",
        regex=True
    )
    upd_dt.loc[mask_na] = pd.to_datetime(fixed_2yy, errors="coerce")

df["Updated_dt"] = upd_dt
df = df.dropna(subset=["Updated_dt"])
df["quarter"] = df["Updated_dt"].dt.to_period("Q")

# --- 2) Keep rows where at least one of the flag columns contains "Yes" ---
present_flags = [c for c in flag_cols if c in df.columns]
if not present_flags:
    raise KeyError(f"None of the flag columns are present: {flag_cols}")

yes_any = (
    df[present_flags]
      .astype(str)
      .apply(lambda col: col.str.contains(r"\byes\b", case=False, na=False))
      .any(axis=1)
)
df_yes = df.loc[yes_any, ["quarter", owner_col]].copy()
df_yes[owner_col] = df_yes[owner_col].fillna("Unknown")

# --- 3) Counts per quarter x owner; totals & percentages per quarter ---
counts = (
    df_yes.groupby(["quarter", owner_col])
          .size()
          .reset_index(name="n")
)

# Ensure continuous quarter range (even if 0)
all_quarters = pd.period_range(df["quarter"].min(), df["quarter"].max(), freq="Q")

totals = counts.groupby("quarter", as_index=False)["n"].sum().rename(columns={"n": "q_total"})
pct = counts.merge(totals, on="quarter", how="left")
pct["pct"] = np.where(pct["q_total"] > 0, 100 * pct["n"] / pct["q_total"], 0.0)

# --- 4) Per quarter: keep top 3 Owner units, aggregate remaining to "Other" ---
top3 = (
    pct.sort_values(["quarter", "pct"], ascending=[True, False])
       .groupby("quarter", as_index=False)
       .head(3)
)

top3_sum = top3.groupby("quarter", as_index=False)["pct"].sum().rename(columns={"pct": "top3_pct"})
other = totals.merge(top3_sum, on="quarter", how="left").fillna({"top3_pct": 0})
other["pct"] = (100 - other["top3_pct"]).clip(lower=0)
other = other.loc[other["pct"] > 0, ["quarter", "pct"]]
other[owner_col] = "Other"

plot_df = pd.concat(
    [
        top3[["quarter", owner_col, "pct"]],
        other[["quarter", owner_col, "pct"]],
    ],
    ignore_index=True
)

# --- 5) Pivot to stacked % per quarter ---
pivot = plot_df.pivot_table(
    index="quarter",
    columns=owner_col,
    values="pct",
    aggfunc="sum",
    fill_value=0,
)

pivot = pivot.reindex(all_quarters, fill_value=0)

# Put "Other" last; sort remaining by total contribution
cols = list(pivot.columns)
if "Other" in cols:
    non_other = [c for c in cols if c != "Other"]
    non_other_sorted = pivot[non_other].sum().sort_values(ascending=False).index.tolist()
    col_order = non_other_sorted + ["Other"]
else:
    col_order = pivot.sum().sort_values(ascending=False).index.tolist()
pivot = pivot[col_order]

# --- 6) Plot: stacked bars with UNIQUE colors per Owner unit (and matching legend) ---
quarters = list(pivot.index.astype(str))
x = np.arange(len(quarters))

fig, ax = plt.subplots(figsize=(14, 6))

# Create a stable unique color for each Owner unit (no reuse)
units = list(pivot.columns)
n_series = len(units)
cmap = plt.cm.get_cmap("tab20", n_series) if n_series <= 20 else plt.cm.get_cmap("hsv", n_series)
unit_colors = {u: cmap(i) for i, u in enumerate(units)}

bottom = np.zeros(len(pivot), dtype=float)
legend_handles = []
for u in units:
    vals = pivot[u].values
    bars = ax.bar(x, vals, bottom=bottom, label=u, color=unit_colors[u])
    bottom += vals
    # use the first bar from this series for the legend, so colors match exactly
    legend_handles.append(bars[0])

ax.set_xticks(x)
ax.set_xticklabels(quarters, rotation=45, ha="right")
ax.set_ylabel("Percent of Cases with A Quote")
ax.set_title('Top 3 Business Units in Percentage per Quarter')
ax.set_ylim(0, 100)
ax.grid(True, axis="y", alpha=0.3)
ax.margins(x=0.02)

# Legend outside; uses the exact colored handles (no color reuse)
plt.subplots_adjust(right=0.82, bottom=0.2)
ax.legend(handles=legend_handles, labels=units, title="BU",
          loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=True)

plt.show()



## Case Status Analysis

### Accounts

In [ ]:
# --- Columns ---
updated_col = "Updated"
status_col  = "Status & health"
acct_col    = "Account"

# --- 1) Working copy + robust date parsing (Updated → quarter) ---
df = tracker.copy()

upd_clean = (
    df[updated_col].astype(str)
      .str.replace("\u00a0", " ", regex=False)
      .str.replace("\r", " ", regex=False)
      .str.replace("\n", " ", regex=False)
      .str.strip()
)

upd_dt = pd.to_datetime(upd_clean, errors="coerce")
mask_na = upd_dt.isna()
if mask_na.any():
    # fallback: m/d/yy -> m/d/20yy (e.g., 9/5/25 -> 9/5/2025)
    fixed_2yy = upd_clean[mask_na].str.replace(
        r"^(\d{1,2})/(\d{1,2})/(\d{2})$",
        r"\1/\2/20\3",
        regex=True
    )
    upd_dt.loc[mask_na] = pd.to_datetime(fixed_2yy, errors="coerce")

df["Updated_dt"] = upd_dt
df = df.dropna(subset=["Updated_dt"])
df["quarter"] = df["Updated_dt"].dt.to_period("Q")

# --- 2) Keep rows where Status & health is COMPLETE / COMPLETED ---
is_complete = df[status_col].astype(str).str.fullmatch(r"complete(d)?", case=False, na=False)
dfc = df.loc[is_complete, ["quarter", acct_col]].copy()
dfc[acct_col] = dfc[acct_col].fillna("Unknown")

# --- 2a) Lump INT, INP, INP/U30 into single category "INT/INP/U30" ---
acct_norm = dfc[acct_col].astype(str).str.strip()
mask_combo = acct_norm.str.upper().isin({"INT", "INT-INP"})
dfc.loc[mask_combo, acct_col] = "INT/INP"

if dfc.empty:
    print("No COMPLETE rows found.")
else:
    # --- 3) Count COMPLETE rows per quarter × account (with combined bucket) ---
    counts = (
        dfc.groupby(["quarter", acct_col])
           .size()
           .reset_index(name="n")
    )

    # Continuous quarter index (based on COMPLETE rows)
    all_quarters = pd.period_range(counts["quarter"].min(), counts["quarter"].max(), freq="Q")

    # Totals per quarter
    totals = counts.groupby("quarter", as_index=False)["n"].sum().rename(columns={"n": "q_total"})

    # --- 4) Per quarter: keep top 3 accounts, group the rest as "Other" ---
    top3 = (
        counts.sort_values(["quarter", "n"], ascending=[True, False])
              .groupby("quarter", as_index=False)
              .head(3)
    )

    top3_sum = top3.groupby("quarter", as_index=False)["n"].sum().rename(columns={"n": "top3_n"})
    other = totals.merge(top3_sum, on="quarter", how="left").fillna({"top3_n": 0})
    other["n"] = (other["q_total"] - other["top3_n"]).clip(lower=0)
    other = other.loc[other["n"] > 0, ["quarter", "n"]]
    other[acct_col] = "Other"

    plot_df = pd.concat(
        [top3[["quarter", acct_col, "n"]], other[["quarter", acct_col, "n"]]],
        ignore_index=True
    )

    # --- 5) Pivot to quarter × (top3 accounts + Other) in COUNTS ---
    pivot = plot_df.pivot_table(
        index="quarter",
        columns=acct_col,
        values="n",
        aggfunc="sum",
        fill_value=0
    ).reindex(all_quarters, fill_value=0)

    # Put "Other" last; sort remaining accounts by total count
    cols = list(pivot.columns)
    if "Other" in cols:
        non_other = [c for c in cols if c != "Other"]
        non_other_sorted = pivot[non_other].sum().sort_values(ascending=False).index.tolist()
        col_order = non_other_sorted + ["Other"]
    else:
        col_order = pivot.sum().sort_values(ascending=False).index.tolist()
    pivot = pivot[col_order]

    # --- 6) Stacked bar plot (counts) ---
    quarters = list(pivot.index.astype(str))
    x = np.arange(len(quarters))

    fig, ax = plt.subplots(figsize=(14, 6))
    bottom = np.zeros(len(pivot), dtype=float)

    # stable unique colors per account
    accounts = list(pivot.columns)
    n_series = len(accounts)
    cmap = plt.cm.get_cmap("tab20", n_series) if n_series <= 20 else plt.cm.get_cmap("hsv", n_series)
    acct_colors = {a: cmap(i) for i, a in enumerate(accounts)}

    handles = []
    for a in accounts:
        vals = pivot[a].values
        bars = ax.bar(x, vals, bottom=bottom, label=a, color=acct_colors[a])
        bottom += vals
        handles.append(bars[0])

    ax.set_xticks(x)
    ax.set_xticklabels(quarters, rotation=45, ha="right")
    ax.set_ylabel("Number of Complete Cases")
    ax.set_title('Top 3 Accounts by Number of Complete Cases per Quarter')
    ax.grid(True, axis="y", alpha=0.3)
    ax.margins(x=0.02)

    # Legend outside
    plt.subplots_adjust(right=0.82, bottom=0.2)
    ax.legend(handles=handles, labels=accounts, title="Account",
              loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=True)

    plt.show()


### Main Product Lines

In [ ]:
# --- Columns ---
prod_col    = 'Product\nline'
updated_col = 'Updated'
status_col  = 'Status & health'

# --- 1) Working copy ---
df = tracker.copy()

# --- 2) Map Product line to 6 top-level groups by prefix ---
pl_norm = df[prod_col].astype(str).str.strip().str.lower()
df['Product group'] = np.select(
    [
        pl_norm.str.startswith('event'),
        pl_norm.str.startswith('viewables'),
        pl_norm.str.startswith('home'),
        pl_norm.str.startswith('others'),
        pl_norm.str.startswith('wearable'),
        pl_norm.str.startswith('hearable'),
    ],
    ['Event', 'Viewables', 'Home', 'Others', 'Wearable', 'Hearable'],
    default=np.nan
)
df = df.dropna(subset=['Product group'])

# --- 3) Parse Updated -> datetime robustly, then to quarter ---
upd_clean = (df[updated_col].astype(str)
             .str.replace("\u00a0", " ", regex=False)
             .str.replace("\r", " ", regex=False)
             .str.replace("\n", " ", regex=False)
             .str.strip())
upd_dt = pd.to_datetime(upd_clean, errors='coerce')
mask_na = upd_dt.isna()
if mask_na.any():
    fixed_2yy = upd_clean[mask_na].str.replace(
        r'^(\d{1,2})/(\d{1,2})/(\d{2})$', r'\1/\2/20\3', regex=True
    )
    upd_dt.loc[mask_na] = pd.to_datetime(fixed_2yy, errors='coerce')

df['Updated_dt'] = upd_dt
df = df.dropna(subset=['Updated_dt'])
df['quarter'] = df['Updated_dt'].dt.to_period('Q')

# --- 4) Keep rows where Status & health equals COMPLETE (allow "Completed" too) ---
is_complete = df[status_col].astype(str).str.fullmatch(r'complete(d)?', case=False, na=False)
dfc = df.loc[is_complete, ['quarter', 'Product group']].copy()

if dfc.empty:
    print('No COMPLETE rows found.')
else:
    # --- 5) Count COMPLETE rows per quarter × product group ---
    counts = (dfc.groupby(['quarter', 'Product group'])
                   .size()
                   .unstack('Product group', fill_value=0))

    # Ensure continuous quarter range (even if some quarters have zero)
    all_quarters = pd.period_range(counts.index.min(), counts.index.max(), freq='Q')
    counts = counts.reindex(all_quarters, fill_value=0)

    # Order columns in a fixed logical order (use what exists)
    desired_order = ['Event', 'Viewables', 'Home', 'Others', 'Wearable', 'Hearable']
    present = [c for c in desired_order if c in counts.columns]
    counts = counts[present]

    # --- 6) Stacked bar plot (counts per group) ---
    quarters = list(counts.index.astype(str))
    x = np.arange(len(quarters))

    fig, ax = plt.subplots(figsize=(14, 6))

    # Optional: stable unique colors for the six groups
    palette = {
        'Event':    '#1f77b4',
        'Viewables':'#ff7f0e',
        'Home':     '#2ca02c',
        'Others':   '#9467bd',
        'Wearable': '#8c564b',
        'Hearable': '#e377c2',
    }
    bottom = np.zeros(len(counts), dtype=float)
    handles = []
    labels  = []
    for col in counts.columns:
        vals = counts[col].values
        bars = ax.bar(x, vals, bottom=bottom, label=col, color=palette.get(col, None))
        bottom += vals
        handles.append(bars[0])
        labels.append(col)

    ax.set_xticks(x)
    ax.set_xticklabels(quarters, rotation=45, ha='right')
    ax.set_ylabel('Number of Complete Cases')
    ax.set_title('Quarterly Number of Complete Cases by Main Product Lines')
    ax.grid(True, axis='y', alpha=0.3)
    ax.margins(x=0.02)

    # Legend outside
    plt.subplots_adjust(right=0.82, bottom=0.2)
    ax.legend(handles=handles, labels=labels, title='Product Line',
              loc='center left', bbox_to_anchor=(1.02, 0.5), frameon=True)

    plt.show()

## Large Language Model

In [ ]:
suspended.to_csv("suspended.csv", index=False)

In [ ]:
! python -m pip install --upgrade openai python-dotenv

In [ ]:
! python azure_gpt_csv.py

## Machine Learning

In [ ]:
ml = tracker[tracker["Status & health"] != "DUPLICATE"]
mapping = {
    "COMPLETE": "COMPLETE",
    "SUSPENDED": "INCOMPLETE",
    "LOST": "INCOMPLETE"
}

ml["classes"] = ml["Status & health"].map(mapping)

# --- 1) Split data into features and target ---
X = ml[["Time \n(days)"]]   # replace with the actual column name
y = ml["classes"]

# --- 2) Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0, stratify=y
)

# --- 3) Define and fit Logistic Regression model ---
model = LogisticRegression()
model.fit(X_train, y_train)

# --- 4) Predictions ---
y_pred = model.predict(X_test)

pos_class = "COMPLETE"
y_prob = model.predict_proba(X_test)[:, list(model.classes_).index(pos_class)]

# Compute ROC
fpr, tpr, thresholds = roc_curve(y_test, y_prob, pos_label=pos_class)
auc_score = roc_auc_score((y_test == pos_class).astype(int), y_prob)

# Plot
plt.plot(fpr, tpr, label=f"AUC = {auc_score:.3f}")
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title(f"ROC-AUC Curve Using Time to Predict Status")
plt.legend()
plt.show()